In [4]:
import cv2
import numpy as np
import dlib
import time


def extract_index_nparray(nparray):
    index = None
    for num in nparray[0]:
        index = num
        break
    return index


source = "C:/BLOG/Computer_Vision_Project/Computer_Vision_Project/face_swapping_in_8_steps/bradley_cooper.jpg"
target = "C:/BLOG/Computer_Vision_Project/Computer_Vision_Project/face_swapping_in_8_steps/jim_carrey.jpg"

# 读取两种面部图片
img = cv2.imread(source)
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
mask = np.zeros_like(img_gray)

img2 = cv2.imread(target)
img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

cv2.imshow("source", img)
cv2.imshow("img_gray", img_gray)

cv2.imshow("target", img2)

cv2.waitKey(0)
cv2.destroyAllWindows()


In [5]:
# dlib 是人脸检测相关的库
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(
    r"F:\jupyter\Pytorch\computer_vision\projects\classification\pytorch\expression_project\deploy_demo\face_detector_trained\shape_predictor_68_face_landmarks.dat"
)
height, width, channels = img2.shape
img2_new_face = np.zeros((height, width, channels), np.uint8)

In [6]:
# Face 1
faces = detector(img_gray)  # 人脸检测，

# #--------------------------------#
# for face in faces:
# #     print()
#     pt1 = (face.left(), face.top())
#     pt2 = ( face.right(), face.bottom())
#     cv2.rectangle(img, pt1, pt2, (0, 255, 255))
# # cv2.imshow("img", img)
# # cv2.waitKey(0)
# # cv2.destroyAllWindows()
# #------------#------------------#

In [9]:
for face in faces:
    landmarks = predictor(img_gray, face)  # 关键点检测
    landmarks_points = []
    for n in range(0, 68):
        # 关键点的坐标
        x = landmarks.part(n).x  
        y = landmarks.part(n).y
        landmarks_points.append((x, y))

    points = np.array(landmarks_points, np.int32)
#     #----------------#----------------#
#     # 绘制出关键点
#     for i, point in enumerate(points):
#         center = (point[0], point[1])
#         font = cv2.FONT_HERSHEY_SIMPLEX  # 指定字体
#         cv2.putText(img, str(i), center, font, 0.5, (255, 255, 255), 1, cv2.LINE_AA)  # 绘制的图像，文字，文字左下角的坐标,字体，字体颜色，厚度等
#         cv2.circle(img, center, 3, (255, 255, 0))
#     #---------------#-----------------#

    # 凸包检测
    convexhull = cv2.convexHull(points)
#     #--------------------------------#
#     # 根据凸包检测的结果绘制凸包轮廓
#     hullcount = convexhull.shape[0]
#     for i in range(hullcount-1):
#         x0 = convexhull[i][0][0]
#         y0 = convexhull[i][0][1]
#         x1 = convexhull[i+1][0][0]
#         y1 = convexhull[i+1][0][1]
#         cv2.line(img, (x0, y0), (x1, y1), 255, 2)
#         if i == hullcount-2:
#             x2 = convexhull[0][0][0]
#             y2 = convexhull[0][0][1]
#             cv2.line(img, (x1, y1), (x2, y2), 255, 2)    

#     cv2.imshow("dst", img)    
#     #--------------------------------#

    # cv2.polylines(img, [convexhull], True, (255, 0, 0), 3)
    # 填充凸包生成的多边形
    cv2.fillConvexPoly(mask, convexhull, 255)
    # 获得脸部的区域，其他皆为黑色背景
    face_image_1 = cv2.bitwise_and(img, img, mask=mask)
#     #--------------------------------#    
#     cv2.imshow("mask", mask)
#     cv2.imshow("face_image_1", face_image_1)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
#     #--------------------------------#

    # Delaunay triangulation
    # 矩形框
    rect = cv2.boundingRect(convexhull)
    subdiv = cv2.Subdiv2D(rect)
    subdiv.insert(landmarks_points)
    triangles = subdiv.getTriangleList()
    triangles = np.array(triangles, dtype=np.int32)
    indexes_triangles = []
    for t in triangles:
        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])

        index_pt1 = np.where((points == pt1).all(axis=1))
        index_pt1 = extract_index_nparray(index_pt1)

        index_pt2 = np.where((points == pt2).all(axis=1))
        index_pt2 = extract_index_nparray(index_pt2)

        index_pt3 = np.where((points == pt3).all(axis=1))
        index_pt3 = extract_index_nparray(index_pt3)

        if index_pt1 is not None and index_pt2 is not None and index_pt3 is not None:
            triangle = [index_pt1, index_pt2, index_pt3]
            indexes_triangles.append(triangle)

[[171 436 100 463 129 421]
 [100 463 171 436 105 513]
 [273 770 312 671 324 779]
 [312 671 273 770 286 668]
 [305 523 344 545 325 552]
 [344 545 305 523 301 490]
 [115 562 105 513 190 442]
 [344 545 301 490 372 440]
 [126 612 115 562 217 608]
 [285 611 309 613 286 646]
 [309 613 285 611 308 604]
 [150 660 126 612 217 608]
 [286 646 309 613 311 650]
 [187 701 150 660 217 608]
 [415 732 338 667 372 647]
 [338 667 415 732 374 768]
 [228 738 187 701 251 649]
 [263 548 250 603 217 608]
 [250 603 263 548 283 554]
 [273 770 228 738 286 668]
 [263 548 301 490 305 523]
 [301 490 263 548 240 439]
 [374 768 324 779 338 667]
 [253 386 293 427 240 439]
 [293 427 253 386 320 387]
 [451 689 415 732 372 647]
 [396 417 372 440 369 418]
 [372 440 396 417 397 440]
 [481 642 451 689 401 601]
 [216 441 217 419 240 439]
 [217 419 216 441 191 419]
 [496 590 481 642 401 601]
 [191 419 216 441 190 442]
 [500 537 496 590 401 601]
 [191 419 145 390 179 374]
 [145 390 191 419 171 436]
 [500 485 500 537 397 440]
 

In [10]:
# Face 2
faces2 = detector(img2_gray)
for face in faces2:
    landmarks = predictor(img2_gray, face)
    landmarks_points2 = []
    for n in range(0, 68):
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        landmarks_points2.append((x, y))


    points2 = np.array(landmarks_points2, np.int32)
    convexhull2 = cv2.convexHull(points2)

lines_space_mask = np.zeros_like(img_gray)
lines_space_new_face = np.zeros_like(img2)
# Triangulation of both faces
for triangle_index in indexes_triangles:
    # Triangulation of the first face
    tr1_pt1 = landmarks_points[triangle_index[0]]
    tr1_pt2 = landmarks_points[triangle_index[1]]
    tr1_pt3 = landmarks_points[triangle_index[2]]
    triangle1 = np.array([tr1_pt1, tr1_pt2, tr1_pt3], np.int32)


    rect1 = cv2.boundingRect(triangle1)
    (x, y, w, h) = rect1
    cropped_triangle = img[y: y + h, x: x + w]
    cropped_tr1_mask = np.zeros((h, w), np.uint8)


    points = np.array([[tr1_pt1[0] - x, tr1_pt1[1] - y],
                       [tr1_pt2[0] - x, tr1_pt2[1] - y],
                       [tr1_pt3[0] - x, tr1_pt3[1] - y]], np.int32)

    cv2.fillConvexPoly(cropped_tr1_mask, points, 255)

    # Lines space
    cv2.line(lines_space_mask, tr1_pt1, tr1_pt2, 255)
    cv2.line(lines_space_mask, tr1_pt2, tr1_pt3, 255)
    cv2.line(lines_space_mask, tr1_pt1, tr1_pt3, 255)
    lines_space = cv2.bitwise_and(img, img, mask=lines_space_mask)

    # Triangulation of second face
    tr2_pt1 = landmarks_points2[triangle_index[0]]
    tr2_pt2 = landmarks_points2[triangle_index[1]]
    tr2_pt3 = landmarks_points2[triangle_index[2]]
    triangle2 = np.array([tr2_pt1, tr2_pt2, tr2_pt3], np.int32)


    rect2 = cv2.boundingRect(triangle2)
    (x, y, w, h) = rect2

    cropped_tr2_mask = np.zeros((h, w), np.uint8)

    points2 = np.array([[tr2_pt1[0] - x, tr2_pt1[1] - y],
                        [tr2_pt2[0] - x, tr2_pt2[1] - y],
                        [tr2_pt3[0] - x, tr2_pt3[1] - y]], np.int32)

    cv2.fillConvexPoly(cropped_tr2_mask, points2, 255)

    # Warp triangles
    points = np.float32(points)
    points2 = np.float32(points2)
    M = cv2.getAffineTransform(points, points2)
    warped_triangle = cv2.warpAffine(cropped_triangle, M, (w, h))
    warped_triangle = cv2.bitwise_and(warped_triangle, warped_triangle, mask=cropped_tr2_mask)

    # Reconstructing destination face
    img2_new_face_rect_area = img2_new_face[y: y + h, x: x + w]
    img2_new_face_rect_area_gray = cv2.cvtColor(img2_new_face_rect_area, cv2.COLOR_BGR2GRAY)
    _, mask_triangles_designed = cv2.threshold(img2_new_face_rect_area_gray, 1, 255, cv2.THRESH_BINARY_INV)
    warped_triangle = cv2.bitwise_and(warped_triangle, warped_triangle, mask=mask_triangles_designed)

    img2_new_face_rect_area = cv2.add(img2_new_face_rect_area, warped_triangle)
    img2_new_face[y: y + h, x: x + w] = img2_new_face_rect_area



# Face swapped (putting 1st face into 2nd face)
img2_face_mask = np.zeros_like(img2_gray)
img2_head_mask = cv2.fillConvexPoly(img2_face_mask, convexhull2, 255)
img2_face_mask = cv2.bitwise_not(img2_head_mask)


img2_head_noface = cv2.bitwise_and(img2, img2, mask=img2_face_mask)
result = cv2.add(img2_head_noface, img2_new_face)

(x, y, w, h) = cv2.boundingRect(convexhull2)
center_face2 = (int((x + x + w) / 2), int((y + y + h) / 2))

seamlessclone = cv2.seamlessClone(result, img2, img2_head_mask, center_face2, cv2.NORMAL_CLONE)

cv2.imshow("seamlessclone", seamlessclone)
cv2.waitKey(0)



cv2.destroyAllWindows()
